<a href="https://colab.research.google.com/github/AntoineYK/tp-colab/blob/main/tp1_antoineMerle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TP: PySpark pour analyse de données de films.

Initialiser la session Spark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MoviesData").getOrCreate()

Charger le fichier CSV dans un DataFrame

In [ ]:
file_path = "Rotten Tomatoes Movies.csv"
df = spark.read.option("header", "true").csv(file_path)

df.show(10)

### Supprimer les lignes contenant des valeurs nulles

In [ ]:
df_clean = df.dropna()

df_clean.show(10)

### Convertir les colonnes in_theaters_date et on_streaming_date au format yyyy-mm-dd

In [ ]:
from pyspark.sql.functions import col, to_date

In [ ]:
df_clean = df_clean \
    .withColumn("in_theaters_date", to_date(col("in_theaters_date"), "yyyy-mm-dd")) \
    .withColumn("on_streaming_date", to_date(col("on_streaming_date"), "yyyy-mm-dd")) \
    .na.drop(subset=["in_theaters_date", "on_streaming_date"])

In [ ]:
df_clean.show(10)

+--------------------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+----------------+-----------------+------------------+--------------------+------------------+------------------+-----------------+---------------+--------------+
|         movie_title|          movie_info|   critics_consensus|rating|               genre|           directors|             writers|                cast|in_theaters_date|on_streaming_date|runtime_in_minutes|         studio_name|tomatometer_status|tomatometer_rating|tomatometer_count|audience_rating|audience_count|
+--------------------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+----------------+-----------------+------------------+--------------------+------------------+------------------+-----------------+---------------+--------------+
|Percy Jackson & t...|A teenager discov...|Tho

### Filtre des films avec une note inférieure à 20

In [ ]:
df_filtered = df_clean.filter(col("tomatometer_rating") < 20)

df_under20 = df_filtered.orderBy("tomatometer_rating", ascending=True)

df_under20.show()

+--------------------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+----------------+-----------------+------------------+--------------------+------------------+------------------+-----------------+---------------+--------------+
|         movie_title|          movie_info|   critics_consensus|rating|               genre|           directors|             writers|                cast|in_theaters_date|on_streaming_date|runtime_in_minutes|         studio_name|tomatometer_status|tomatometer_rating|tomatometer_count|audience_rating|audience_count|
+--------------------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+----------------+-----------------+------------------+--------------------+------------------+------------------+-----------------+---------------+--------------+
|National Lampoon'...|American independ...|It 

### Filtre des films sortis après l'année 2000

In [ ]:
df_films_after_2000 = df_clean.filter(col("in_theaters_date") > "2000-01-01")

df_after2000 = df_films_after_2000.orderBy("in_theaters_date", ascending=True)

# Afficher les résultats
df_after2000.show()

+--------------------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+----------------+-----------------+------------------+--------------------+------------------+------------------+-----------------+---------------+--------------+
|         movie_title|          movie_info|   critics_consensus|rating|               genre|           directors|             writers|                cast|in_theaters_date|on_streaming_date|runtime_in_minutes|         studio_name|tomatometer_status|tomatometer_rating|tomatometer_count|audience_rating|audience_count|
+--------------------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+----------------+-----------------+------------------+--------------------+------------------+------------------+-----------------+---------------+--------------+
|           Supernova|Supernova chronic...|Thi

### La note moyenne des films par studio

In [ ]:
from pyspark.sql.functions import avg, round

In [ ]:
df_avg_studio = df_clean.groupBy("studio_name").agg(
    avg("tomatometer_rating").alias("average_tomatometer_rating")
)

#Afficher les résultats
df_avg_studio.show()

+--------------------+--------------------------+
|         studio_name|average_tomatometer_rating|
+--------------------+--------------------------+
|    Relativity Media|         33.32142857142857|
|  New World Pictures|         69.66666666666667|
|Alluvial Film Com...|                      92.0|
|       Shout Factory|                      65.0|
|            El Deseo|                      84.0|
|Oscilloscope Pict...|         80.36842105263158|
|       Cavu Pictures|                      72.5|
|        Toho Company|                      86.0|
|  Fine Line Features|        60.785714285714285|
|           HBO Video|         74.55555555555556|
|             42 West|                      50.0|
|   Empire Film Group|                      15.0|
|        Disneynature|                      77.0|
| Perdido Productions|                      71.0|
|  Big World Pictures|                      77.5|
|          Wellspring|                      67.6|
|       October Films|                      82.8|


### La note moyenne des films par réalisateur (arrondi au 10ème)

In [ ]:
df_avg_studio = df_clean.groupBy("directors").agg(
    round(avg("tomatometer_rating"), 1).alias("average_tomatometer_rating")
)

#Afficher les résultats
df_avg_studio.show()

+--------------------+--------------------------+
|           directors|average_tomatometer_rating|
+--------------------+--------------------------+
|    Laurence Olivier|                      91.0|
|        Jim Jarmusch|                      74.6|
|          John Wells|                      53.3|
|Harry Elfont, Deb...|                      40.0|
|         John Milius|                      66.0|
|          Will Gluck|                      59.3|
|          Rob Bowman|                      26.5|
|       Paul Morrison|                      24.0|
|        Michael Kang|                      87.0|
|Molly Bingham, St...|                      82.0|
|       Carlos Brooks|                      61.0|
|      Chan-wook Park|                      77.6|
|       Greg Pritikin|                      71.0|
|        Zak Hilditch|                      86.0|
|Jon Hurwitz, Hayd...|                      45.0|
|    Peter Strickland|                      89.5|
|       Peter Sattler|                      75.0|


### Diviser les genres multiples d’une colonne en genres individuels

In [ ]:
from pyspark.sql.functions import split, explode, col, ceil

In [ ]:
df_indiv_genres = df_clean \
    .withColumn("genre", explode(split(col("genre"), ",\s*")))

df_indiv_genres.show()

+--------------------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+----------------+-----------------+------------------+--------------------+------------------+------------------+-----------------+---------------+--------------+
|         movie_title|          movie_info|   critics_consensus|rating|               genre|           directors|             writers|                cast|in_theaters_date|on_streaming_date|runtime_in_minutes|         studio_name|tomatometer_status|tomatometer_rating|tomatometer_count|audience_rating|audience_count|
+--------------------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+----------------+-----------------+------------------+--------------------+------------------+------------------+-----------------+---------------+--------------+
|Percy Jackson & t...|A teenager discov...|Tho

### Calculer la durée moyenne des films pour chaque genre

In [ ]:
average_runtime_by_genre = df_indiv_genres \
    .groupBy("genre") \
    .agg(ceil(avg(col("runtime_in_minutes").cast("float"))).alias("average_runtime")) \
    .orderBy("average_runtime", ascending=False)

average_runtime_by_genre.show()

+--------------------+---------------+
|               genre|average_runtime|
+--------------------+---------------+
|            Classics|            116|
|             Western|            116|
|Faith & Spirituality|            112|
|  Action & Adventure|            110|
|               Drama|            110|
|             Romance|            109|
|Art House & Inter...|            108|
|Science Fiction &...|            108|
|  Mystery & Suspense|            107|
|Musical & Perform...|            106|
|    Sports & Fitness|            105|
|         Cult Movies|            102|
|              Comedy|            101|
|       Anime & Manga|            100|
|          Television|            100|
|       Gay & Lesbian|            100|
|    Special Interest|             99|
|              Horror|             99|
|         Documentary|             97|
|       Kids & Family|             96|
+--------------------+---------------+
only showing top 20 rows

